In [34]:
import pandas as pd

review_datas = pd.read_csv("./data/Musical_instruments_reviews.csv")

In [35]:
review_datas.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [39]:
review_datas = review_datas.loc[:,['reviewText', 'summary']]
review_datas = review_datas.dropna(axis=0)
review_datas.head(5)

,reviewText,summary
0,"Not much to write about here, but it does exac...",good
1,The product does exactly as it should and is q...,Jake
2,The primary job of this device is to block the...,It Does The Job Well
3,Nice windscreen protects my MXL mic and preven...,GOOD WINDSCREEN FOR THE MONEY
4,This pop filter is great. It looks and perform...,No more pops when I record my vocals.


In [40]:
review_datas['reviewText'] = review_datas['reviewText'].str.lower()
review_datas['reviewText'] = review_datas['reviewText'].str.replace('[^\w]', ' ')
review_datas['summary'] = review_datas['summary'].str.lower()
review_datas['summary'] = review_datas['summary'].str.replace('[^\w]', ' ')

In [41]:
review_datas.head(5)

,reviewText,summary
0,not much to write about here but it does exac...,good
1,the product does exactly as it should and is q...,jake
2,the primary job of this device is to block the...,it does the job well
3,nice windscreen protects my mxl mic and preven...,good windscreen for the money
4,this pop filter is great it looks and perform...,no more pops when i record my vocals


In [44]:
encoder_input, decoder_input, decoder_output = [], [], []

for stc in review_datas['reviewText']:
    encoder_input.append(stc.split())
    
for stc in review_datas['summary']:
    decoder_input.append(("<start> " + stc).split())

for stc in review_datas['summary']:
    decoder_output.append((stc + " <end>").split())

In [46]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:
tokenizer_re = Tokenizer()
tokenizer_re.fit_on_texts(encoder_input)
encoder_input = tokenizer_re.texts_to_sequences(encoder_input)

tokenizer_su = Tokenizer()
tokenizer_su.fit_on_texts(decoder_input)
tokenizer_su.fit_on_texts(decoder_output)
decoder_input = tokenizer_su.texts_to_sequences(decoder_input)
decoder_output = tokenizer_su.texts_to_sequences(decoder_output)

In [48]:
su_to_index = tokenizer_su.word_index
index_to_su = tokenizer_su.index_word

In [49]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

In [52]:
encoder_input.shape, decoder_input.shape, decoder_output.shape

((10254, 2089), (10254, 27), (10254, 27))

In [53]:
test_size = 2500
encoder_input_train = encoder_input[:-test_size]
decoder_input_train = decoder_input[:-test_size]
decoder_output_train = decoder_output[:-test_size]

encoder_input_test = encoder_input[-test_size:]
decoder_input_test = decoder_input[-test_size:]
decoder_output_test = decoder_output[-test_size:]

In [54]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [55]:
encoder_inputs = Input(shape=(2089,))
encoder_embed = Embedding(len(tokenizer_re.word_index)+1, 50)(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(50, return_state=True, return_sequences=True)(encoder_mask)

In [57]:
decoder_inputs = Input(shape=(27,))
decoder_embed = Embedding(len(tokenizer_re.word_index)+1, 50)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])

In [ ]:
#### MODEL

In [ ]:
encoder_h_state = Input(shape=(50,))
encoder_c_state = Input(shape=(50,))

pd_decoder_outputs, pd_h_state, pd_c_state = decoder_lstm(decoder_mask, initial_state=[encoder_h_state, encoder_c_state])

### 어텐션

pd_encoder_outputs = Input(shape=(2089, 50))
